# Remote crawler database (client)

When we have the server part running, we should use the implemented REST endpoints to communicate with our database. We use the *requests* library to implement a client.

First we will set up parameters and disable warning messages (our SSL certificate is not signed by a trusted authority).

In [15]:
import requests

In [16]:
AUTH = ("Crawler1", "BestNonencryptedPasswordEver!!!")
ENDPOINT = "https://127.0.0.1:5000"

In [17]:
print(requests.get(ENDPOINT + "/db/get_values", verify = False, auth = AUTH).json())

[{'1': 1194}, {'2': 1634}]


Disable warnings and try again

In [18]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [19]:
print(requests.get(ENDPOINT + "/db/get_values", verify = False, auth = AUTH).json())

[{'1': 1194}, {'2': 1634}]


### Showcase example

First we define the two methods that will be called in parallel by two groups of three workers. Each method will call its own endpoint to increase a counter value in a database.

In [20]:
def increase_db_values(counter_id, increases):
    for i in range(increases):
        requests.post(ENDPOINT + f"/db/increase/{counter_id}", verify = False, auth = AUTH)
        
def increase_db_values_locking(counter_id, increases):
    for i in range(increases):
        requests.post(ENDPOINT + f"/db/increase_locking/{counter_id}", verify = False, auth = AUTH)

Now we run the client program (e.g. one multi-threaded instance of a crawler) as initially presented in the *Web crawling - basic tools* notebook.

In [21]:
import concurrent.futures

# reset_db_values
requests.post(ENDPOINT + "/db/reset", verify = False, auth = AUTH).json()
# print_db_values
print(requests.get(ENDPOINT + "/db/get_values", verify = False, auth = AUTH).json())

with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    print(f"\n ... executing workers ...\n")
    for _ in range(3):
        executor.submit(increase_db_values, 1,1000)
    for _ in range(3):
        executor.submit(increase_db_values_locking, 2,1000)
    
# print_db_values
print(requests.get(ENDPOINT + "/db/get_values", verify = False, auth = AUTH).json())

[{'1': 0}, {'2': 0}]

 ... executing workers ...

[{'1': 2211}, {'2': 3000}]


You should get an output similar to the following:
    
> [{'1': 0}, {'2': 0}]
> 
>  ... executing workers ...
>
> [{'1': 2211}, {'2': 3000}]